In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)


2.1.0


In [2]:
from data_processing import import_and_clean_train, label_encoding
import numpy as np


train_X, test_X, train_y, test_y, label_dict = import_and_clean_train('train', encoder=label_encoding)

train_X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,499,20,3,65.0,7800,1,3,3,0,4,...,0,0,0,0,0,0,6,2009,8,4
1,9,50,4,51.0,6120,1,3,3,0,4,...,0,205,0,0,0,0,4,2008,8,0
2,310,20,3,90.0,12378,1,0,3,0,4,...,175,0,0,0,0,0,11,2006,8,4
3,958,20,3,70.0,7420,1,3,3,0,4,...,0,0,0,0,0,0,4,2007,8,4
4,1023,50,4,52.0,9439,1,3,3,0,4,...,0,192,0,0,0,0,3,2007,8,4


In [3]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_X))


AttributeError: module 'tensorflow_core.keras.layers' has no attribute 'Normalization'

In [ ]:
GrLivArea = np.array(train_X['GrLivArea'])

GrLivArea_normalizer = layers.Normalization(input_shape=[1,], axis=None)
GrLivArea_normalizer.adapt(GrLivArea)

GrLivArea_model = tf.keras.Sequential([
    GrLivArea_normalizer,
    layers.Dense(units=1)
])

GrLivArea_model.summary()
